In [1]:
# choose your data set:
sim_data_path = 'Traffic_light_sim_only'
real_data_path = 'Traffic_light_real_only'
both_data_path = 'Traffic_light_both'

def get_validation_path(path):
    return path + '/test_images'

In [2]:
# choose your data set:
train_data_path = both_data_path

validation_data_path = get_validation_path(train_data_path)




from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Dropout, Convolution2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

batch_size = 64

nb_epoch = 125
nb_classes = 4
image_shape = [64, 64, 3]

model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=image_shape))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# save best model only 
# NOTE: needs validation_data in fit_generator or validation_split in ImageDataGenerator, which isn't available
model_file_name = train_data_path + '_' + 'light_classifier_model.h5'
checkpointer = ModelCheckpoint(monitor='val_acc', filepath=model_file_name, save_best_only=True, verbose=1)

# reduce learning rate when loss doesn't improve much anymore (to pin-point the local minima)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', epsilon=0.03, factor=0.4, patience=4, min_lr=0.001, verbose=1)

# stop when there was no real improvement anymore (even after reducing the learning rate several times)
early_stop = EarlyStopping(monitor='val_acc', min_delta=0.01, patience=12, verbose=1)


# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

datagen = ImageDataGenerator(width_shift_range=.2, height_shift_range=.2, shear_range=0.05, zoom_range=.2,
                             fill_mode='nearest', rescale=1. / 255)

image_train_data_gen = datagen.flow_from_directory(train_data_path, target_size=(64, 64), batch_size=batch_size,
                                             classes=['red', 'yellow', 'green', 'none'])

image_valid_data_gen = datagen.flow_from_directory(validation_data_path, target_size=(64, 64), batch_size=batch_size,
                                             classes=['red', 'yellow', 'green', 'none'])

model.fit_generator(image_train_data_gen, nb_epoch=nb_epoch,
                    samples_per_epoch=image_train_data_gen.nb_sample,
                    validation_data=image_valid_data_gen,
                    nb_val_samples=image_valid_data_gen.nb_sample,
                    callbacks=[checkpointer,reduce_lr, early_stop])

# model.save('light_classifier_model.h5')

Using TensorFlow backend.


Found 5016 images belonging to 4 classes.
Found 390 images belonging to 4 classes.
Epoch 1/125
5016/5016 [==============================] - 75s - loss: 1.0846 - acc: 0.5789 - val_loss: 1.1617 - val_acc: 0.4205
Epoch 2/125
5016/5016 [==============================] - 70s - loss: 0.8137 - acc: 0.6663 - val_loss: 1.1134 - val_acc: 0.5385
Epoch 3/125
5016/5016 [==============================] - 69s - loss: 0.7046 - acc: 0.7163 - val_loss: 1.2579 - val_acc: 0.4692
Epoch 4/125
5016/5016 [==============================] - 82s - loss: 0.6374 - acc: 0.7448 - val_loss: 0.8201 - val_acc: 0.6282
Epoch 5/125
5016/5016 [==============================] - 72s - loss: 0.5704 - acc: 0.7715 - val_loss: 0.8910 - val_acc: 0.5897
Epoch 6/125
5016/5016 [==============================] - 69s - loss: 0.5263 - acc: 0.7787 - val_loss: 1.0930 - val_acc: 0.6103
Epoch 7/125
5016/5016 [==============================] - 70s - loss: 0.5064 - acc: 0.7994 - val_loss: 0.6744 - val_acc: 0.7308
Epoch 8/125
5016/5016 [=====

5016/5016 [==============================] - 63s - loss: 0.2251 - acc: 0.9197 - val_loss: 0.2770 - val_acc: 0.9103
Epoch 31/125
5016/5016 [==============================] - 62s - loss: 0.2069 - acc: 0.9193 - val_loss: 0.3275 - val_acc: 0.8744
Epoch 32/125
5016/5016 [==============================] - 61s - loss: 0.2211 - acc: 0.9213 - val_loss: 0.3144 - val_acc: 0.8769
Epoch 33/125
5016/5016 [==============================] - 62s - loss: 0.2067 - acc: 0.9244 - val_loss: 0.3516 - val_acc: 0.8410
Epoch 34/125
5016/5016 [==============================] - 61s - loss: 0.2328 - acc: 0.9256 - val_loss: 0.2980 - val_acc: 0.8846
Epoch 35/125
5016/5016 [==============================] - 62s - loss: 0.2017 - acc: 0.9258 - val_loss: 0.3158 - val_acc: 0.8769
Epoch 36/125
5016/5016 [==============================] - 61s - loss: 0.2037 - acc: 0.9308 - val_loss: 0.2797 - val_acc: 0.9000
Epoch 37/125
5016/5016 [==============================] - 66s - loss: 0.1949 - acc: 0.9286 - val_loss: 0.2699 - val_a

In [3]:
model.load_weights(filepath=model_file_name)

datagen = ImageDataGenerator(fill_mode='nearest', rescale=1. / 255)  # no randomization

data_path = sim_data_path
train_data_gen = datagen.flow_from_directory(data_path, target_size=(64, 64), batch_size=batch_size,
                                             classes=['red', 'yellow', 'green', 'none'])
valid_data_gen = datagen.flow_from_directory(get_validation_path(data_path), target_size=(64, 64), batch_size=batch_size,
                                             classes=['red', 'yellow', 'green', 'none'])

print("Simulation Training loss and accuracy: ", model.evaluate_generator(train_data_gen, train_data_gen.nb_sample))
print("Simulation Validation loss and accuracy: ", model.evaluate_generator(valid_data_gen, valid_data_gen.nb_sample))


data_path = real_data_path
train_data_gen = datagen.flow_from_directory(data_path, target_size=(64, 64), batch_size=batch_size,
                                             classes=['red', 'yellow', 'green', 'none'])
valid_data_gen = datagen.flow_from_directory(get_validation_path(data_path), target_size=(64, 64), batch_size=batch_size,
                                             classes=['red', 'yellow', 'green', 'none'])

print("RealWorld Training loss and accuracy: ", model.evaluate_generator(train_data_gen, train_data_gen.nb_sample))
print("RealWorld Validation loss and accuracy: ", model.evaluate_generator(valid_data_gen, valid_data_gen.nb_sample))

Found 2165 images belonging to 4 classes.
Found 200 images belonging to 4 classes.
('Simulation Training loss and accuracy: ', [0.054847969111339052, 0.97921478131626827])
('Simulation Validation loss and accuracy: ', [0.06379626348614692, 0.96999999999999997])
Found 2851 images belonging to 4 classes.
Found 190 images belonging to 4 classes.
('RealWorld Training loss and accuracy: ', [0.075042060402266728, 0.97544721152564173])
('RealWorld Validation loss and accuracy: ', [0.19421776329216203, 0.92631578006242454])
